In [1]:
import numpy as np 
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(color_codes = True)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/home/hf/anaconda2/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dataset = pd.read_csv('4classdata.csv')
dataset.tail()

,MMSI,LONGITUDE,LATITUDE,speedmin,accmin2,avgspeed,meanacc,stdspeed,date,COARSE_FIS,rowNum
2194298,87,116.328900,39.978650,0.073591,0.001958,0.114921,0.217491,0.159637,2007-08-20 00:49:53,2,2194299
2194299,87,116.329033,39.977917,0.086664,0.013761,0.114921,0.217491,0.159637,2007-08-20 00:50:50,2,2194300
2194300,87,116.329650,39.977517,0.061650,-0.022401,0.114921,0.217491,0.159637,2007-08-20 00:51:57,2,2194301
2194301,87,116.330250,39.977283,0.076442,0.019724,0.076442,0.019724,0.000000,2007-08-20 00:52:42,2,2194302
2194302,87,116.330683,39.976417,0.078380,0.001472,0.078380,0.001472,0.000000,2007-08-20 00:54:01,2,2194303


In [3]:
dataset['date'] = pd.to_datetime(dataset['date'])    
dataset['date_delta'] = (dataset['date'] - dataset['date'].min())  / np.timedelta64(1,'D')
dataset = dataset.drop(['date'],1)
dataset.head()

,MMSI,LONGITUDE,LATITUDE,speedmin,accmin2,avgspeed,meanacc,stdspeed,COARSE_FIS,rowNum,date_delta
0,10,86.186028,41.741415,0.026938,-0.014162,0.891573,0.017554,0.393636,1,1,350.126308
1,10,86.179470,41.737063,0.740528,0.725685,0.891573,0.017554,0.393636,1,2,350.126991
2,10,86.172823,41.734105,0.642173,-0.098355,0.891573,0.017554,0.393636,1,3,350.127685
3,10,86.166563,41.739110,0.774170,0.134234,1.170899,0.002555,0.574789,1,4,350.128368
4,10,86.159987,41.744368,0.813248,0.039741,1.170899,0.002555,0.574789,1,5,350.129051


In [4]:
uni_person = dataset.MMSI.unique()
uni_person

array([10, 20, 21, 52, 58, 60, 62, 64, 65, 67, 68, 69, 73, 75, 76, 78, 80,
       81, 82, 84, 85, 86, 87])

In [5]:
train_mmsi = np.random.choice(uni_person, size = 20,replace = False)
train_mmsi = np.sort(train_mmsi)

In [6]:
df = pd.DataFrame()
for i in train_mmsi: 
    df = df.append(dataset.loc[dataset['MMSI']==i])

In [7]:
df.groupby('MMSI').size()

MMSI
10    131494
20    128148
52    139295
60        16
62    266249
64     37390
65     71440
67     50596
68    661833
69      4193
73     10592
75      5812
76       960
78     36569
80       629
81     37131
82     20552
84    236797
85    350404
86       559
dtype: int64

In [14]:
#specify the people --> sequence len for each person 
seq_len=df.groupby('MMSI').size().values
sum(seq_len)

2190659

In [9]:
labels = np.asarray(df['COARSE_FIS']).astype(np.float32)
labels

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [10]:
features = np.asarray(df.drop(['MMSI','COARSE_FIS','rowNum'],1).values).astype(np.float32)
n_samples, num_features = features.shape
num_features

8

In [11]:
train_x, train_y = [],[]
seq_size = 250
for i in range(len(features)-seq_size-1):
    train_x.append(features[i:i+seq_size])
    train_y.append(labels[i:i+seq_size])

2190408